In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
import os

warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd


* Data: aggregate infections / test consumption for all policies, all pooling methods, dilution only
* Use: Figure 4, 5, 6 (Pareto plot and zoom-ins)
* Format: pandas DataFrame

In [3]:
import pickle

NUM_TRIALS = 200

param_config = {
    "country": ["US"],

    "pop_size_default": 10000,

    "init_prev": [0.01],
    "init_prev_default": 0.01,

    'horizon_default': 100,

    "num_groups_default": 5,
    "num_groups": [1,2,3,4,5,6,7],

    'pool_size': [5,10,15,20],
    'pool_size_default': 10,

    'LoD_default': 1240,
    'LoD': [1240],

    "edge_weight_default": 10,
    "edge_weight": [10],

    "alpha_default": 5.0,
    "alpha": [2.0],

    "peak_VL": [6.0],

    "distancing_scale": [50.0],

    "dilute": ["average"],

    "beta_default": 0.1, # transmissibility
    "sigma_default": 0.2, # rate E --> I_pre
    "lamda_default": 0.5, # rate I_pre --> I_(a)sym
    "gamma_default": 0.25 # rate I_sym --> R
}


In [4]:
import itertools

param_values = {}

for param in [
        "country", "pop_size", "init_prev", "num_groups", "pool_size", "horizon", 
        "beta", "sigma", "lamda", "gamma", "LoD", "edge_weight", "alpha", "peak_VL",
        "distancing_scale", "dilute"
    ]:
        if param in param_config:
            param_values[param] = param_config[param]
        else:
            param_values[param] = [param_config[param+"_default"]]

all_param_configs = [dict(zip(param_values.keys(), x)) for x in itertools.product(*param_values.values())]

In [5]:
import copy
from collections import defaultdict


all_data_dilution = []

for param_config_single in all_param_configs:

    if param_config_single["dilute"] != "average":
        continue

    traj_info = [param_config_single, {}]
    
    path = f"/home/yz685/corr_pooling_seirsplus/results/{param_config_single['country']}"
    for param in [
        "pop_size", "init_prev", "num_groups", "pool_size", "horizon", 
        "beta", "sigma", "lamda", "gamma", "LoD", "edge_weight", "alpha", "peak_VL", "distancing_scale", "dilute"
    ]:
        path += f"_{param}={param_config_single[param]}"

    data = copy.deepcopy(param_config_single)

    results = defaultdict(list)

    for pooling_strategy in ["naive", "correlated", "correlated_weak"]:
        for i in range(1, NUM_TRIALS+1):
            filepath = path + f"/{pooling_strategy}/results_{i}.pickle"
            try:
                with open(filepath, "rb") as f:
                    results[pooling_strategy].append(pickle.load(f))
            except Exception as e:
                print("error: ", e)
                continue

    if len(results["naive"]) == 0:
        continue

    for metric in ["cumInfections", "cum_num_tests"]:
        
        for pooling_strategy in ["naive", "correlated", "correlated_weak"]:

            results_tmp = []
            for SEED in range(1,NUM_TRIALS+1):
                try:
                    results_tmp.append(
                        [x[metric]\
                        for x in results[pooling_strategy][SEED]]
                    )
                except:
                    pass

            results_tmp_ = np.array([(xi+[xi[-1]]*(100-len(xi)))[:100] for xi in results_tmp], dtype=float)
            mean = np.nanmean(results_tmp_, axis=0)
            sem = np.nanstd(results_tmp_, axis=0)/np.sqrt(NUM_TRIALS)
            traj_info[1][metric+"_"+pooling_strategy] = [mean, sem] # TODO: confirm

            if metric in ["cumInfections", "cum_num_tests"]:
                data[f"{metric}_{pooling_strategy}_mean"] = mean[-1]
                data[f"{metric}_{pooling_strategy}_sem"] = sem[-1]
            else:
                data[f"{metric}_{pooling_strategy}_mean"] = np.nanmean(mean)
                data[f"{metric}_{pooling_strategy}_sem"] = np.nanmean(sem)

    all_data_dilution.append(data)


In [6]:
df = pd.DataFrame(all_data_dilution)
df = df[
    (df["init_prev"] == 0.01)
    & (df["distancing_scale"] == 50)
    & (df["alpha"] == 2.0)
    & (df["dilute"] == "average")
]


In [7]:
df.drop(
    columns = [
        'country', 'pop_size', 'init_prev', 
        'horizon', 'beta', 'sigma', 'lamda', 'gamma', 'LoD', 'edge_weight',
        'alpha', 'peak_VL', 'distancing_scale', 'dilute'
    ],
    inplace=True
)

In [8]:
df

,num_groups,pool_size,cumInfections_naive_mean,cumInfections_naive_sem,cumInfections_correlated_mean,cumInfections_correlated_sem,cumInfections_correlated_weak_mean,cumInfections_correlated_weak_sem,cum_num_tests_naive_mean,cum_num_tests_naive_sem,cum_num_tests_correlated_mean,cum_num_tests_correlated_sem,cum_num_tests_correlated_weak_mean,cum_num_tests_correlated_weak_sem
0,1,5,2051.251256,20.948657,2036.974874,20.938199,2026.618090,19.709068,204713.658291,46.884009,202338.507538,24.620735,203310.673367,31.903768
1,1,10,2159.271357,22.040444,2111.959799,20.553085,2130.849246,20.224624,117401.698492,172.379207,111139.562814,106.466337,113476.331658,125.851888
2,1,15,2198.236181,20.309189,2136.145729,21.116896,2158.502513,19.970672,94480.914573,245.832145,84284.015075,169.396782,87949.507538,189.785725
3,1,20,2244.809045,20.194822,2164.708543,20.589415,2184.130653,20.747775,87538.919598,320.688229,73405.060302,213.569509,78299.080402,255.659501
4,2,5,2135.326633,21.721077,2090.291457,21.263999,2129.708543,21.175840,107648.492462,76.645796,104466.020101,46.439610,105830.763819,57.271161
5,2,10,2248.648241,20.880999,2166.512563,21.818692,2218.497487,20.935744,69359.472362,173.992836,61889.949749,118.284114,64592.914573,135.319101
6,2,15,2348.668342,20.659370,2203.502513,20.323019,2249.326633,22.130061,63490.336683,251.096354,51096.366834,161.095621,55032.989950,203.466489
7,2,20,2348.005025,21.637944,2264.125628,19.272534,2283.231156,18.934727,64053.422111,331.098113,48336.175879,194.222546,53337.763819,224.074098
8,3,5,2370.517588,22.416997,2250.291457,20.596024,2312.889447,20.358772,76187.638191,88.330823,72178.236181,49.897944,73855.768844,62.376560
9,3,10,2470.949749,21.664425,2303.929648,22.219851,2365.155779,21.396123,54812.924623,179.365668,46196.608040,124.179896,49334.296482,140.482771


In [9]:
df.to_csv("../../data/infections_testconsumption_dilution.csv", index=False)

* Data: aggregate infections for all policies, NP and CCP, and all test error models
* Use: Figure 7 and Figure EC3 (comparing NP inf - CP inf for different test error models)
* Format: pickled dictionaries

In [10]:
import pickle

NUM_TRIALS = 200

param_config = {
    "country": ["US"],

    "pop_size_default": 10000,

    "init_prev": [0.01],
    "init_prev_default": 0.01,

    'horizon_default': 100,

    "num_groups_default": 5,
    "num_groups": [1,2,3,4,5,6,7],

    'pool_size': [5,10,15,20],
    'pool_size_default': 10,

    'LoD_default': 1240,
    'LoD': [1240],

    "edge_weight_default": 10,
    "edge_weight": [10],

    "alpha_default": 5.0,
    "alpha": [2.0],

    "peak_VL": [6.0],

    "distancing_scale": [50.0],

    "dilute": ["average", "sum", "constant_0.5", "constant_0.7", "constant_1.0"],

    "beta_default": 0.1, # transmissibility
    "sigma_default": 0.2, # rate E --> I_pre
    "lamda_default": 0.5, # rate I_pre --> I_(a)sym
    "gamma_default": 0.25 # rate I_sym --> R
}

import itertools

param_values = {}

for param in [
        "country", "pop_size", "init_prev", "num_groups", "pool_size", "horizon", 
        "beta", "sigma", "lamda", "gamma", "LoD", "edge_weight", "alpha", "peak_VL",
        "distancing_scale", "dilute"
    ]:
        if param in param_config:
            param_values[param] = param_config[param]
        else:
            param_values[param] = [param_config[param+"_default"]]

all_param_configs = [dict(zip(param_values.keys(), x)) for x in itertools.product(*param_values.values())]

In [11]:
import copy
from collections import defaultdict
import numpy as np


final_inf_np = {}
final_inf_cp = {}

for param_config_single in all_param_configs:
    
    path = f"/home/yz685/corr_pooling_seirsplus/results/{param_config_single['country']}"
    for param in [
        "pop_size", "init_prev", "num_groups", "pool_size", "horizon", 
        "beta", "sigma", "lamda", "gamma", "LoD", "edge_weight", "alpha", "peak_VL", "distancing_scale", "dilute"
    ]:
        path += f"_{param}={param_config_single[param]}"

    results = defaultdict(list)

    for pooling_strategy in ["naive", "correlated_weak"]:
        for i in range(1, NUM_TRIALS+1):
            filepath = path + f"/{pooling_strategy}/results_{i}.pickle"
            try:
                with open(filepath, "rb") as f:
                    results[pooling_strategy].append(pickle.load(f))
            except Exception as e:
                print("error: ", e)
                continue

    if len(results["naive"]) == 0:
        continue

    cumInfections_by_trial = {}

    for metric in ["cumInfections"]:
        
        for pooling_strategy in ["naive", "correlated_weak"]:

            results_tmp = []
            for seed_idx in range(NUM_TRIALS):
                try:
                    results_tmp.append(
                        [x[metric]\
                        for x in results[pooling_strategy][seed_idx]]
                    )
                except:
                    pass

            results_tmp_ = np.array([(xi+[xi[-1]]*(100-len(xi)))[:100] for xi in results_tmp], dtype=float)
            if metric == "cumInfections":
                cumInfections_by_trial[pooling_strategy] = results_tmp_[:,-1]
    
    final_inf_np[(param_config_single["num_groups"], param_config_single["pool_size"], param_config_single["dilute"])] = cumInfections_by_trial["naive"]
    final_inf_cp[(param_config_single["num_groups"], param_config_single["pool_size"], param_config_single["dilute"])] = cumInfections_by_trial["correlated_weak"]

# save final_inf_np, final_inf_cp
with open("../../data/infections_np_nodil.pickle", "wb") as f:
    pickle.dump(final_inf_np, f)
with open("../../data/infections_ccp_nodil.pickle", "wb") as f:
    pickle.dump(final_inf_cp, f)

* Data: detailed trajectory info for (5,10) under the dilution model
* Use: Figure 3
* Format: pickled nested list-dictionary

In [12]:
import pickle

NUM_TRIALS = 200

param_config = {
    "country": ["US"],

    "pop_size_default": 10000,

    "init_prev": [0.01],
    "init_prev_default": 0.01,

    'horizon_default': 100,

    "num_groups_default": 5,
    "num_groups": [5],

    'pool_size': [10],
    'pool_size_default': 10,

    'LoD_default': 1240,
    'LoD': [1240],

    "edge_weight_default": 10,
    "edge_weight": [10],

    "alpha_default": 5.0,
    "alpha": [2.0],

    "peak_VL": [6.0],

    "distancing_scale": [50.0],

    "dilute": ["average"],

    "beta_default": 0.1, # transmissibility
    "sigma_default": 0.2, # rate E --> I_pre
    "lamda_default": 0.5, # rate I_pre --> I_(a)sym
    "gamma_default": 0.25 # rate I_sym --> R
}

import itertools

param_values = {}

for param in [
        "country", "pop_size", "init_prev", "num_groups", "pool_size", "horizon", 
        "beta", "sigma", "lamda", "gamma", "LoD", "edge_weight", "alpha", "peak_VL",
        "distancing_scale", "dilute"
    ]:
        if param in param_config:
            param_values[param] = param_config[param]
        else:
            param_values[param] = [param_config[param+"_default"]]

all_param_configs = [dict(zip(param_values.keys(), x)) for x in itertools.product(*param_values.values())]

In [13]:
import copy
from collections import defaultdict


trajectories = [] # list of tuples (dict of param_config_single, dict of {metric: [mean, std]})

for param_config_single in all_param_configs:

    traj_info = [param_config_single, {}]
    
    path = f"/home/yz685/corr_pooling_seirsplus/results/{param_config_single['country']}"
    for param in [
        "pop_size", "init_prev", "num_groups", "pool_size", "horizon", 
        "beta", "sigma", "lamda", "gamma", "LoD", "edge_weight", "alpha", "peak_VL", "distancing_scale", "dilute"
    ]:
        path += f"_{param}={param_config_single[param]}"

    results = defaultdict(list)

    for pooling_strategy in ["naive", "correlated", "correlated_weak"]:
        for i in range(1, NUM_TRIALS+1):
            filepath = path + f"/{pooling_strategy}/results_{i}.pickle"
            try:
                with open(filepath, "rb") as f:
                    results[pooling_strategy].append(pickle.load(f))
            except Exception as e:
                # print("error: ", e)
                continue

    if len(results["naive"]) == 0:
        continue

    metrics = set(results["correlated"][0][0].keys()) - set(["day", "VL_in_positive_pools"])

    for metric in metrics:
        
        for pooling_strategy in ["naive", "correlated", "correlated_weak"]:

            results_tmp = []
            for SEED in range(NUM_TRIALS):
                try:
                    results_tmp.append(
                        [x[metric]\
                        for x in results[pooling_strategy][SEED]]
                    )
                except:
                    pass

            results_tmp_ = np.array([(xi+[xi[-1]]*(100-len(xi)))[:100] for xi in results_tmp], dtype=float)
            mean = np.nanmean(results_tmp_, axis=0)
            sem = np.nanstd(results_tmp_, axis=0)/np.sqrt(NUM_TRIALS)
            traj_info[1][metric+"_"+pooling_strategy] = [mean, sem] 

    trajectories.append(traj_info)

with open("../../data/trajectory_info_numgroups=5_poolsize=10.pickle", "wb") as f:
    pickle.dump(trajectories, f)
